# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [10]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

## API KEY 불러오기

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [12]:
load_dotenv()

True

## 대화 기록 함수 정의

In [13]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [14]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 내 이름은 양인규야.")],
    config=config,
)

print(response.content)

안녕하세요, 양인규님! 만나서 반갑습니다. 어떻게 도와드릴까요?


## 두 번째 대화

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 양인규입니다! 도움이 필요하시거나 궁금한 것이 있으면 말씀해 주세요.


## 세션 테스트

In [16]:
config = {"configurable": {"session_id": "abc3"}}  # 다른 세션 ID로 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 저는 당신의 이름을 알지 못합니다. 당신의 이름을 알려주시면 좋겠습니다!'

In [17]:
config = {"configurable": {"session_id": "abc2"}}  # 다시 처음 세션 ID로 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 양인규라는 것에 대해 이야기했습니다. 그 외에 더 궁금한 것이나 다른 주제로 이야기하고 싶은 것이 있으면 말씀해 주세요!'

## 스트리밍 응답 출력

In [18]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")  # 토큰 단위로 구분하기 위해 '|' 문자 사용

양인규라는 이름은 한국에서 흔히 쓰이는 이름입니다. 그렇다면 한국 사람일 가능성이 높겠네요! 

한국 문화에 대해 간단히 말씀드리자면:

1. **음식**: 한국 음식은 다양하고 매운 음식이 많습니다. 김치, 불고기, 비빔밥 등이 유명합니다. 또한, 다양한 찌개와 전통적인 간식도 사랑받고 있습니다.

2. **전통**: 한국의 전통 의상인 한복은 특별한 날에 착용합니다. 설날이나 추석 등의 명절에 많이 볼 수 있습니다.

3. **예절**: 한국에서는 공경과 예의가 중요한 가치입니다. 어른에게 존경을 표하고, 기본적인 예절을 중시하는 문화가 있습니다.

4. **K-Pop과 K-드라마**: 최근 몇 년간 한국의 대중문화가 세계적으로 큰 인기를 얻었고, K-Pop, K-드라마가 많은 팬을 끌어모으고 있습니다.

5. **명절**: 설날과 추석은 가장 중요한 한국의 명절로, 가족과 함께 모여 조상을 기리고 음식을 나누는 전통이 있습니다.

혹시 다른 특정 주제에 대해 알고 싶으신 점이 있으시면 말씀해 주세요!